In [ ]:
!pip install langchain langchain[ollama] langchain[community]

In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_ollama import ChatOllama
import os

# Schema for structured response
class Person(BaseModel):
    name: str = Field(description="The person's name", required=True)
    height: float = Field(description="The person's height", required=True)
    hair_color: str = Field(description="The person's hair color")


# Prompt template
prompt = PromptTemplate.from_template(
    """Alex is 5 feet tall. 
Claudia is 1 feet taller than Alex and jumps higher than him. 
Claudia is a brunette and Alex is blonde.

Human: {question}
AI: """
)

# Chain
llm = ChatOllama(model=os.getenv("OPENAI_DEFAULT_MODEL"), format="json", temperature=0)
structured_llm = llm.with_structured_output(Person)
chain = prompt | structured_llm

/home/ubuntu/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
alex = chain.invoke("Describe Alex")
alex

Person(name='Alex', height=5.0, hair_color='blonde')

In [3]:
claudia = chain.invoke("Describe Claudia")
claudia

Person(name='Claudia', height=5.01, hair_color='brunette')

In [4]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

In [5]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class Multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

In [6]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model=os.getenv("OPENAI_DEFAULT_MODEL"), format="json", temperature=0)
llm_with_tools = llm.bind_tools(tools=[Add, Multiply])

query = "what is 11 + 49? Also, What is 3 * 12?"
response = llm_with_tools.invoke(query)
print(response)

content='' additional_kwargs={} response_metadata={} id='run-e131320e-c7aa-4349-bc54-286e6bccdc11-0' tool_calls=[{'name': 'Add', 'args': {'a': 11, 'b': 49}, 'id': 'f95f99e2-fcb4-428d-88a4-e94a872a3cac', 'type': 'tool_call'}]


In [7]:
response.tool_calls

[{'name': 'Add',
  'args': {'a': 11, 'b': 49},
  'id': 'f95f99e2-fcb4-428d-88a4-e94a872a3cac',
  'type': 'tool_call'}]

In [8]:
from langchain_core.messages import HumanMessage, FunctionMessage

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(query)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = str(selected_tool.invoke(tool_call["args"]))
    messages.append(FunctionMessage(tool_output, name=tool_call["name"]))
    
messages

[HumanMessage(content='what is 11 + 49? Also, What is 3 * 12?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={}, id='run-0c3f4efe-c55e-4e21-a6d7-d3372051ff0b-0', tool_calls=[{'name': 'Add', 'args': {'a': 11, 'b': 49}, 'id': '4abeebf2-445f-44d2-8dcb-97089a70f006', 'type': 'tool_call'}]),
 FunctionMessage(content='60', additional_kwargs={}, response_metadata={}, name='Add')]

In [9]:
from langchain_community.llms import Ollama
import os

Ollama(model=os.getenv("OPENAI_DEFAULT_MODEL"), temperature=0).invoke(messages)

/tmp/ipykernel_46080/1621789022.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  Ollama(model=os.getenv("OPENAI_DEFAULT_MODEL"), temperature=0).invoke(messages)


"You asked for the results of two mathematical operations:\n\n1. **11 + 49** = 60\n2. **3 * 12** = 36\n\nLet me know if you have any other calculations you'd like assistance with!"

In [10]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def process_query(query):
    messages = []
    ai_msg = llm_with_tools.invoke(query)
    messages.append(ai_msg)
    for tool_call in ai_msg.tool_calls:
        selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
        tool_output = str(selected_tool.invoke(tool_call["args"]))
        messages.append(FunctionMessage(tool_output, name=tool_call["name"]))   

    chat_prompt = ChatPromptTemplate.from_template("""다음 계산에 근거하여 질문에 간단히 한국어로 답하세요:
                                                {context}
                                                
                                                Question:{question}
                                                Answer:""")
    chat_model = ChatOllama(model=os.getenv("OPENAI_DEFAULT_MODEL"), format="json", temperature=0)
    llm_chain = chat_prompt | chat_model| StrOutputParser()
    response = llm_chain.invoke({"context":messages, "question":query})
    if response:
        return response
    else:
        return "No response found."

query = "23 더하기 17은 얼마입니까?"
process_query(query)

/tmp/ipykernel_46080/2285454035.py:19: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  chat_model = ChatOllama(model=os.getenv("OPENAI_DEFAULT_MODEL"), format="json", temperature=0)


'{\n"답변": "40"\n}\n\n\n   \n\n\n   \n\n\n\n\n\n\n\n\n\t \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'